<a href="https://colab.research.google.com/github/mostafaqapil/InternIntelligence_XGB-Titanic-Optuna/blob/main/Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.7 MB/s eta 0:00:00


In [79]:
import optuna
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from re import S

In [57]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [58]:
df.isnull().sum()

,0
survived,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2
class,0
who,0


In [59]:
df.drop(columns=['deck'],inplace=True)
df.drop(columns=['embark_town'],inplace=True)

df['age'].fillna(df['age'].median())

df['embarked'].fillna(df['embarked'].mode()[0])


,embarked
0,S
1,C
2,S
3,S
4,S
...,...
886,S
887,S
888,S
889,C


In [60]:
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df['embarked'] = df['embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df['class'] = df['class'].map({'First': 0, 'Second': 1, 'Third': 2})
df['who'] = df['who'].map({'man': 0, 'woman': 1, 'child': 2})
df['adult_male'] = df['adult_male'].map({True: 1, False: 0})
df['alive'] = df['alive'].map({'yes': 1, 'no': 0})
df['alone'] = df['alone'].map({True: 1, False: 0})

In [61]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
0,0,3,0,22.0,1,0,7.2500,0.0,2,0,1,0,0
1,1,1,1,38.0,1,0,71.2833,1.0,0,1,0,1,0
2,1,3,1,26.0,0,0,7.9250,0.0,2,1,0,1,1
3,1,1,1,35.0,1,0,53.1000,0.0,0,1,0,1,0
4,0,3,0,35.0,0,0,8.0500,0.0,2,0,1,0,1


In [65]:
print(df.dtypes)

survived         int64
pclass           int64
sex              int64
age            float64
sibsp            int64
parch            int64
fare           float64
embarked       float64
class         category
who              int64
adult_male       int64
alive            int64
alone            int64
dtype: object


In [67]:
df['class'] = df['class'].astype(int)

In [68]:
x = df.drop(columns=['survived'],axis=1)
y = df['survived']

In [69]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(f'train set size: {x_train.shape[0]}')
print(f'test set size: {x_test.shape[0]}')

train set size: 712
test set size: 179


In [81]:
def objective(trial):
    param = {
        'max_depth' : trial.suggest_int('max_depth',3,10),
        'learning_rate' : trial.suggest_float('learning_rate',0.01,0.2),
        'n_estimators' : trial.suggest_int('n_estimators',50,200),
        'subsample' : trial.suggest_float('subsample',0.6,1.0),
        'closample_bytree' : trial.suggest_float('colsample_bytree',0.7,1.0),
        'use_label_encoder' : False,

    }
    model = xgb.XGBClassifier(**param)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, x_train, y_train, cv=skf, scoring='accuracy')
    return score.mean()

In [82]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-04-03 12:06:37,593] A new study created in memory with name: no-name-9a6c1e62-ef25-4725-bdb7-1ee85a208f71
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:06:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "closample_bytree", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:06:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "closample_bytree", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:06:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "closample_bytree", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:06:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "closample_bytree", "use_label_encoder" } are not 

In [83]:
print("Best Hyperparameters:", study.best_params)

Best Hyperparameters: {'max_depth': 5, 'learning_rate': 0.11914322385900246, 'n_estimators': 64, 'subsample': 0.8197157328028154, 'colsample_bytree': 0.9088711699494147}


In [85]:
best_Model = xgb.XGBClassifier(**study.best_params)
best_Model.fit(x_train, y_train)
y_pred = best_Model.predict(x_test)

In [86]:
accuracy = accuracy_score(y_test, y_pred)
print(f' Test Accuracy: {accuracy:.4f}')
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


 Test Accuracy: 1.0000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       105
           1       1.00      1.00      1.00        74

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179

Confusion Matrix:
 [[105   0]
 [  0  74]]
